In [1]:
# feistel_cipher.py
from sbox import apply_sbox, generate_sbox, inverse_sbox
from pbox import apply_pbox
from key_schedule import lfsr_key_schedule


# Perform one round of Feistel encryption.
# :param L: Left 16-bit half
# :param R: Right 16-bit half
# :param subkey: 16-bit round key
# :param sbox: Substitution box
# :return: Updated (L, R)
def feistel_round(L, R, subkey, sbox):
    transformed = apply_sbox(R & 0xF, sbox)  # Apply S-Box on 4-bit parts
    transformed = apply_pbox(transformed)  # Apply P-Box
    transformed ^= subkey  # XOR with subkey

    new_L = R
    new_R = L ^ transformed

    return new_L, new_R

# Encrypt a 32-bit plaintext using Feistel structure.
def encrypt_block(plaintext, key, num_rounds=8):
    L, R = (plaintext >> 16) & 0xFFFF, plaintext & 0xFFFF
    subkeys = lfsr_key_schedule(key, num_rounds)
    sbox = generate_sbox()

    for i in range(num_rounds):
        L, R = feistel_round(L, R, subkeys[i], sbox)

    return (L << 16) | R  # Combine halves


# Decrypt a 32-bit ciphertext using Feistel structure.
def decrypt_block(ciphertext, key, num_rounds=8):
    L, R = (ciphertext >> 16) & 0xFFFF, ciphertext & 0xFFFF
    subkeys = lfsr_key_schedule(key, num_rounds)[::-1]  # Reverse subkeys
    sbox = generate_sbox()
    inv_sbox = inverse_sbox(sbox)

    for i in range(num_rounds):
        L, R = feistel_round(L, R, subkeys[i], inv_sbox)

    return (L << 16) | R  # Combine halves


ModuleNotFoundError: No module named 'sbox'